In [21]:
import boto3
import redis
from botocore.exceptions import NoCredentialsError, PartialCredentialsError
import json

In [12]:
# DynamoDB 및 Redis 클라이언트 초기화
dynamodb = boto3.resource('dynamodb', region_name='ap-northeast-2')
table = dynamodb.Table('merged-data-table')

r = redis.StrictRedis(host='221.151.189.183', port=6379, db=0)

In [22]:
def access_keys(path):
    with open(f'{path}/', 'r') as f:
        key = json.load(f)
    return key

In [9]:
# DynamoDB에서 id 값을 가져오는 함수
def get_ids_from_dynamodb(key):
    AWS_ACCESS_KEY = key['aws_access_key_id']
    AWS_SECRET_KEY = key['aws_secret_key']
    REGION = key['region']
    
    response = table.scan()  # Scan을 통해 DynamoDB 테이블에서 모든 항목을 가져옴
    items = response.get('Items', [])
    return [item['id'] for item in items if 'id' in item]  # 'id' 필드만 추출

# Redis에 id 값을 저장하는 함수
def save_to_redis(ids):
    for id_value in ids:
        if not r.hexists('compare_id', id_value):  # Redis hash에 해당 id가 없으면
            r.hset('compare_id', id_value, 1)  # 해당 id를 해시에 저장
            print(f"새로운 id {id_value}가 Redis에 추가되었습니다.")
        else:
            print(f"id {id_value}는 이미 Redis에 존재합니다.")

In [25]:
def check_dynamodb_connection(key):
    AWS_ACCESS_KEY = key['aws_access_key_id']
    AWS_SECRET_KEY = key['aws_secret_key']
    REGION = key['region']
    
    try:
        dynamodb = boto3.client('dynamodb',
            region_name=REGION,
            aws_access_key_id=AWS_ACCESS_KEY,
            aws_secret_access_key=AWS_SECRET_KEY)
        
        
        # DynamoDB에서 테이블 목록을 가져옴
        response = dynamodb.list_tables()
        table_names = response.get('TableNames', [])
        
        if table_names:
            print(f"DynamoDB에 성공적으로 연결되었습니다. 테이블 목록: {table_names}")
        else:
            print("DynamoDB에 연결되었으나 테이블이 없습니다.")
    except NoCredentialsError:
        print("AWS 자격 증명이 제공되지 않았습니다. 자격 증명을 확인하세요.")
    except PartialCredentialsError:
        print("AWS 자격 증명이 불완전합니다. 자격 증명을 확인하세요.")
    except Exception as e:
        print(f"DynamoDB 연결 중 오류가 발생했습니다: {str(e)}")

In [27]:
def main():
    path = '/home/team3/repository/keys/'
    key = access_keys(path)
    #ids = get_ids_from_dynamodb()
    check_dynamodb_connection(key)
    #save_to_redis(ids)

In [ ]:
if __name__ == "__main__":
    main()